In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append("../")

import gurobipy
from json import dumps, loads
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression as skLogisticRegression
from sklearn.metrics import (classification_report, f1_score, precision_score, recall_score)
from tqdm import tnrange, trange
import tensorflow as tf

from mlsql.influence import InfluenceRanker
from mlsql.fixer import AutoFixer
from mlsql.manager import ModelManagerLM
from mlsql.manager_test import ModelManagerTest

from models.simple_cnn import SimpleCNN
from models.logreg import LogReg
from models.linear_comb import LinearComb
from models.linear_comb_test import LinearCombTest
from processors.breastCancer import BreastCancerCorrProcessor


import logging
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)

import time
import altair as alt
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [2]:
# @tf.function
def rank_fix(ranker, fixer, n):
    rank = ranker.predict()
    fixer.fix(rank, n)
    return rank

@tf.function
def rankit(ranker):
    rank = ranker.predict()
    return rank


@tf.function
def fixit(fixer, rank, n):
    fixer.fix(rank, n)


# @tf.function
def train(manager):
    manager.fit()

In [3]:
# Init
# proc = DiabetesProcessor()
# model = LogReg(1)
# manager0 = ModelManagerLM(proc.x_train, proc.y_train, model, 256)
# start = time.time()
# manager0.fit(print_value=True, max_iter=2000)
# print(time.time() - start)
# manager0.report(proc.x_train, proc.y_train, proc.x_test, proc.y_test)

In [4]:
# Init
proc = BreastCancerCorrProcessor()

In [5]:
model = LogReg(1)
manager0 = ModelManagerLM(proc.x_train, proc.y_corr, model, 256)
start = time.time()
manager0.fit(print_value=True, tol=1e-7, lr=0.5, max_iter=50000)
print(time.time() - start)
manager0.report(proc.x_train, proc.y_corr, proc.x_test, proc.y_test)
# manager0.report(proc.x_train, proc.y_corr, proc.x_query, proc.y_query)

SGD loss: tf.Tensor(0.5153446, shape=(), dtype=float32)
SGD steps: 16903
98.53495836257935
Model name: LogReg
On Training
               precision    recall  f1-score   support

         0.0       0.73      0.87      0.80       309
         1.0       0.54      0.34      0.42       146

    accuracy                           0.70       455
   macro avg       0.64      0.60      0.61       455
weighted avg       0.67      0.70      0.67       455

On Testing
               precision    recall  f1-score   support

         0.0       0.49      1.00      0.66        24
         1.0       1.00      0.24      0.39        33

    accuracy                           0.56        57
   macro avg       0.74      0.62      0.52        57
weighted avg       0.79      0.56      0.50        57



In [51]:
# K = 2600
# corrsel = tf.cast(tf.ones(proc.y_train.shape[0]), dtype='bool')

In [6]:
# K = 87
# corrsel = proc.corrsel
# print(len(list(np.where(corrsel)[0])))
corrsel = proc.corrsel
K = len(list(np.where(corrsel)[0]))
print(len(list(np.where(corrsel)[0])))

146


In [52]:
from tqdm.notebook import tnrange, trange
manager = ModelManagerLM(proc.x_train, proc.y_corr, LogReg(1), 256)
manager.model.set_weights(manager0.model.get_weights())
manager.delta = tf.Variable(manager0.delta.value(), name="delta")
# ranker = InfluenceRanker(manager=manager, on=proc.complain)
from mlsql.loss_ranker import LossRanker
ranker = LossRanker(manager=manager_test)
fixer = AutoFixer(manager, corrsel, K)

AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0
# AQ = proc.complain(manager).AQ
# f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
f1 = f1_score(proc.y_query.numpy(), manager.model.predict(proc.x_query).numpy(), average='weighted')
# AQs.append(float(AQ))
weighted_f1.append(f1)

step_size = 20
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    manager.fit(max_iter=5000, print_value=True, lr=0.1, tol=1e-7)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle

    AQ = proc.complain(manager).AQ
#     f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
    f1 = f1_score(proc.y_query.numpy(), manager.model.predict(proc.x_query).numpy(), average='weighted')
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)
AC = proc.complain(manager).AC

df_rain_test = pd.DataFrame({
#     "Complain": np.array(AQs) - AC,
    "F1": np.array(weighted_f1),
    "K": list(range(0, K, step_size)) + [K],
    "Method": np.repeat("Rain", len(weighted_f1)),
})
alt.Chart(pd.concat([df_rain_test])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain_test.shape[0], grid=False)),
    alt.Y("Complain:Q"),
    color = "Method"
)

  0%|          | 0/8 [00:00<?, ?it/s]

SGD loss: tf.Tensor(0.5193515, shape=(), dtype=float32)
SGD steps: 42
SGD loss: tf.Tensor(0.5194956, shape=(), dtype=float32)
SGD steps: 54
SGD loss: tf.Tensor(0.5169601, shape=(), dtype=float32)
SGD steps: 56
SGD loss: tf.Tensor(0.52369535, shape=(), dtype=float32)
SGD steps: 38
SGD loss: tf.Tensor(0.5278279, shape=(), dtype=float32)
SGD steps: 36
SGD loss: tf.Tensor(0.5305198, shape=(), dtype=float32)
SGD steps: 1
SGD loss: tf.Tensor(0.531772, shape=(), dtype=float32)
SGD steps: 68
SGD loss: tf.Tensor(0.53348625, shape=(), dtype=float32)
SGD steps: 1
Rank_time: 0.08118200302124023
Model_time: 2.1296887397766113


alt.Chart(...)

In [53]:
alt.Chart(pd.concat([df_rain_test])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain_test.shape[0], grid=False)),
    alt.Y("F1:Q"),
    color = "Method"
)

alt.Chart(...)

In [54]:
fixer.recall_k()

array([0.        , 0.        , 0.        , 0.        , 0.00684932,
       0.01369863, 0.02054795, 0.02054795, 0.02054795, 0.02054795,
       0.02054795, 0.02739726, 0.02739726, 0.03424658, 0.03424658,
       0.03424658, 0.04109589, 0.04794521, 0.04794521, 0.04794521,
       0.04794521, 0.04794521, 0.05479452, 0.06164384, 0.06164384,
       0.06164384, 0.06164384, 0.06164384, 0.06164384, 0.06164384,
       0.06164384, 0.06164384, 0.06164384, 0.06849315, 0.07534247,
       0.07534247, 0.07534247, 0.08219178, 0.0890411 , 0.0890411 ,
       0.0890411 , 0.09589041, 0.09589041, 0.09589041, 0.09589041,
       0.10273973, 0.10958904, 0.10958904, 0.10958904, 0.10958904,
       0.10958904, 0.11643836, 0.11643836, 0.12328767, 0.13013699,
       0.1369863 , 0.1369863 , 0.1369863 , 0.1369863 , 0.14383562,
       0.14383562, 0.14383562, 0.14383562, 0.14383562, 0.15068493,
       0.15068493, 0.15068493, 0.15753425, 0.16438356, 0.16438356,
       0.16438356, 0.17123288, 0.17123288, 0.17123288, 0.17123

In [55]:
weighted_f1

[0.4658356417359187,
 0.4658356417359187,
 0.5209103840682788,
 0.5209103840682788,
 0.5209103840682788,
 0.5209103840682788,
 0.5209103840682788,
 0.5209103840682788,
 0.5209103840682788]

In [44]:
# LinearCombTest
model = LinearCombTest(1)
manager_test0 = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_corr, model, 256)
start = time.time()
# tol=1e-7, lr=0.5
manager_test0.fit(print_value=True, tol=1e-10, lr=0.1, max_iter=100000)
print(time.time() - start)
manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_corr, proc.x_a_test, proc.x_b_test, proc.y_test)
# manager_test0.report(proc.x_a_train, proc.x_b_train, proc.y_corr, proc.x_a_query, proc.x_b_query, proc.y_query)

SGD loss: tf.Tensor(0.099993445, shape=(), dtype=float32)
SGD steps: 99999
843.69873046875
Model name: LinearCombTest
On Training
               precision    recall  f1-score   support

         0.0       0.68      1.00      0.81       309
         1.0       0.00      0.00      0.00       146

    accuracy                           0.68       455
   macro avg       0.34      0.50      0.40       455
weighted avg       0.46      0.68      0.55       455

On Testing
               precision    recall  f1-score   support

         0.0       0.42      1.00      0.59        24
         1.0       0.00      0.00      0.00        33

    accuracy                           0.42        57
   macro avg       0.21      0.50      0.30        57
weighted avg       0.18      0.42      0.25        57



In [45]:
corrsel = proc.corrsel
K = len(list(np.where(corrsel)[0]))
print(len(list(np.where(corrsel)[0])))
K

146


146

In [35]:
manager_test = ModelManagerTest(proc.x_a_train, proc.x_b_train, proc.y_corr, LinearCombTest(1), 256)
manager_test.model.set_weights(manager_test0.model.get_weights())
manager_test.delta = tf.Variable(manager_test0.delta.value(), name="delta")
# ranker = InfluenceRanker(manager=manager_test, on=proc.test_complain)
from mlsql.loss_ranker import LossRanker
ranker = LossRanker(manager=manager_test)
fixer = AutoFixer(manager_test, proc.corrsel, K)

# AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_rain = 0
# AQ = proc.test_complain(manager_test).AQ
# f1 = f1_score(proc.y_test.numpy(), manager.model.predict(proc.x_test).numpy(), average='weighted')
f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
# AQs.append(float(AQ))
weighted_f1.append(f1)

step_size = 10
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
    rank = rank_fix(ranker, fixer, nfix)
    middle = time.time()
    # tol=1e-7, lr=0.5
    manager_test.fit(max_iter=100000, tol=1e-10, lr=0.1, print_value=True)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += end - middle

#     AQ = proc.test_complain(manager_test).AQ
    f1 = f1_score(proc.y_test.numpy(), manager_test.model.predict(proc.x_a_test, proc.x_b_test).numpy(), average='weighted')
#     f1 = f1_score(proc.y_query.numpy(), manager_test.model.predict(proc.x_a_query, proc.x_b_query).numpy(), average='weighted')
#     AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Model_time:", model_time_rain)
AC = proc.test_complain(manager_test).AC

df_rain_test = pd.DataFrame({
#     "Complain": np.array(AQs) - AC,
    "F1": np.array(weighted_f1),
    "K": list(range(0, K, step_size)) + [K],
    "Method": np.repeat("Rain", len(weighted_f1)),
})
alt.Chart(pd.concat([df_rain_test])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain_test.shape[0], grid=False)),
    color = "Method"
)

  7%|▋         | 1/15 [01:33<21:54, 93.88s/it]

SGD loss: tf.Tensor(0.10992839, shape=(), dtype=float32)
SGD steps: 11008


 13%|█▎        | 2/15 [15:14<1:52:56, 521.25s/it]

SGD loss: tf.Tensor(0.10094035, shape=(), dtype=float32)
SGD steps: 99999


 20%|██        | 3/15 [28:55<2:11:39, 658.26s/it]

...

 93%|█████████▎| 14/15 [52:26<01:03, 63.16s/it]

SGD loss: tf.Tensor(0.020918112, shape=(), dtype=float32)
SGD steps: 5031


100%|██████████| 15/15 [53:17<00:00, 213.17s/it]

SGD loss: tf.Tensor(0.013923288, shape=(), dtype=float32)
SGD steps: 6361
Rank_time: 0.12161827087402344
Model_time: 3197.3330433368683


NameError: name 'AQs' is not defined

In [43]:
manager_test.report(proc.x_a_train, proc.x_b_train, proc.y_corr, proc.x_a_test, proc.x_b_test, proc.y_test)

Model name: LinearCombTest
On Training
               precision    recall  f1-score   support

         0.0       0.68      1.00      0.81       309
         1.0       0.00      0.00      0.00       146

    accuracy                           0.68       455
   macro avg       0.34      0.50      0.40       455
weighted avg       0.46      0.68      0.55       455

On Testing
               precision    recall  f1-score   support

         0.0       0.42      1.00      0.59        24
         1.0       0.00      0.00      0.00        33

    accuracy                           0.42        57
   macro avg       0.21      0.50      0.30        57
weighted avg       0.18      0.42      0.25        57



/home/yejia/anaconda3/envs/myenv/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [36]:
fixer.recall_k()

array([0.00684932, 0.01369863, 0.02054795, 0.02739726, 0.03424658,
       0.04109589, 0.04794521, 0.05479452, 0.06164384, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849315,
       0.06849315, 0.06849315, 0.06849315, 0.06849315, 0.06849

In [38]:
f1_score(proc.y_test.numpy(), manager_test.model.predict(proc.x_a_test, proc.x_b_test).numpy(), average='weighted')

0.24951267056530213

In [ ]:
# from mlsql.lc_protocol import fit as lc_fit
# from mlsql.lc_protocol import fit_test as lc_fit_test
# from mlsql.lc_protocol import report as lc_report
# from models.linear_comb import LinearComb
# model0_a = LinearComb(1)
# manager0_a = ModelManagerLM(proc.x_a_train, proc.y_train, model0_a, 256)
# model0_b = LinearComb(1)
# manager0_b = ModelManagerLM(proc.x_b_train, proc.y_train, model0_b, 256)
# # enc_time, cpu_time = lc_fit_test(manager0_a, manager0_b, max_iter=2000, tol=1e-6, lr=0.5, print_value=True)
# enc_time, cpu_time = lc_fit(manager0_a, manager0_b, max_iter=1000, tol=1e-6, lr=0.5, print_value=True)
# print(enc_time, cpu_time)
# lc_report(manager0_a, manager0_b, proc.x_a_train, proc.x_b_train, proc.y_train, 
#           proc.x_a_test, proc.x_b_test, proc.y_test)

In [ ]:
from mlsql.lc_protocol import fit as lc_fit
from mlsql.lc_protocol import fit_test as lc_fit_test
from mlsql.lc_protocol import report as lc_report
from models.linear_comb import LinearComb
model0_a = LinearComb(1)
manager0_a = ModelManagerLM(proc.x_a_train, proc.y_corr, model0_a, 256)
model0_b = LinearComb(1)
manager0_b = ModelManagerLM(proc.x_b_train, proc.y_corr, model0_b, 256)
# enc_time, cpu_time = lc_fit_test(manager0_a, manager0_b, max_iter=2000, tol=1e-6, lr=0.5, print_value=True)
enc_time, cpu_time = lc_fit(manager0_a, manager0_b, max_iter=20000, tol=1e-7, lr=0.5, print_value=True)
print(enc_time, cpu_time)
lc_report(manager0_a, manager0_b, proc.x_a_train, proc.x_b_train, proc.y_corr, 
          proc.x_a_test, proc.x_b_test, proc.y_test)
lc_report(manager0_a, manager0_b, proc.x_a_train, proc.x_b_train, proc.y_corr, 
          proc.x_a_query, proc.x_b_query, proc.y_query)

In [ ]:
from mlsql.lc_protocol import f1 as lc_f1
from mlsql.lc_protocol import rank_fix as lc_rank_fix
from mlsql.lc_protocol import complain_value

model_a = LinearComb(1)
manager_a = ModelManagerLM(proc.x_a_train, proc.y_corr, model_a, 256)
manager_a.model.set_weights(model0_a.get_weights())
manager_a.delta = tf.Variable(manager0_a.delta.value(), name="delta")
model_b = LinearComb(1)
manager_b = ModelManagerLM(proc.x_b_train, proc.y_corr, model_b, 256)
manager_b.model.set_weights(model0_b.get_weights())
manager_b.delta = tf.Variable(manager0_b.delta.value(), name="delta")

fixer_a = AutoFixer(manager_a, corrsel, K)
fixer_b = AutoFixer(manager_b, corrsel, K)

AQs = []
weighted_f1 = []
rank_list = []
rank_time_rain = 0
model_time_enc, model_time_rain = 0, 0
query_data = proc.query_data()
AQ = proc.lc_complain(manager_a, manager_b).AQ
# AQ = complain_value(manager_a, manager_b, proc.fl_complain, query_data)
# f1 = lc_f1(manager_a, manager_b, proc.x_a_test, proc.x_b_test, proc.y_test)
f1 = lc_f1(manager_a, manager_b, proc.x_a_query, proc.x_b_query, proc.y_query)
AQs.append(float(AQ))
weighted_f1.append(f1)

step_size = 10
rain_k = int(np.ceil(K / step_size))
for k in trange(0, rain_k):
    nfix = min(step_size, K - step_size * k)
    assert nfix > 0

    start = time.time()
#     rank = lc_rank_fix(fixer_a, fixer_b, nfix, manager_a, manager_b, proc.lc_complain, query_data)
    rank = lc_rank_fix(fixer_a, fixer_b, nfix, manager_a, manager_b, proc.fl_complain, query_data)
    middle = time.time()
#     enc_time, cpu_time = lc_fit_test(manager_a, manager_b, max_iter=1000, tol=1e-6, lr=0.5, print_value=True)
    enc_time, cpu_time = lc_fit(manager_a, manager_b, max_iter=1000, tol=1e-7, lr=0.5, print_value=True)
    end = time.time()
    
    rank_list.append(rank.numpy())
    rank_time_rain += middle - start
    model_time_rain += cpu_time
    model_time_enc += enc_time

    AQ = proc.lc_complain(manager_a, manager_b).AQ
#     AQ = complain_value(manager_a, manager_b, proc.fl_complain, query_data)
#     f1 = lc_f1(manager_a, manager_b, proc.x_a_test, proc.x_b_test, proc.y_test)
    f1 = lc_f1(manager_a, manager_b, proc.x_a_query, proc.x_b_query, proc.y_query)
    AQs.append(float(AQ))
    weighted_f1.append(f1)

print("Rank_time:", rank_time_rain)
print("Retrain_cpu_time:", model_time_rain)
print("Retrain_enc_time:", model_time_enc)

df_rain = pd.DataFrame({
    "Complain": np.array(AQs) - AC,
    "F1": np.array(weighted_f1),
    "K": list(range(0, K, step_size)) + [K],
    "Method": np.repeat("LC_Rain", len(AQs)),
})
alt.Chart(pd.concat([df_rain])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain.shape[0], grid=False)),
    alt.Y("Complain:Q"),
    color = "Method"
)

In [ ]:
AQs

In [31]:
# fixer_a.recall_k()

In [27]:
print(sorted(fixer_a.deletions.numpy()))
print(list(np.where(corrsel)[0]))

[17, 22, 39, 73, 93, 122, 129, 130, 133, 166, 185, 191, 243, 248, 262, 270, 272, 276, 283, 298, 313, 331, 339, 357, 397, 412, 427, 438, 450]
[68, 74, 79, 83, 92, 125, 131, 146, 169, 170, 182, 191, 198, 217, 219, 238, 245, 248, 251, 294, 308, 321, 340, 350, 367, 400, 436, 437, 444]


In [ ]:
alt.Chart(pd.concat([df_rain])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_rain.shape[0], grid=False)),
    alt.Y("F1:Q"),
    color = "Method"
)

In [ ]:
K = 210
step_size = 10
AQs = [-0.16054906056013812, -0.15944249602805974, -0.1580060542769359, -0.15719313225969206, 
       -0.15582570537932483, -0.1567545207834361, -0.15714041809657475, -0.1569623401963131, 
       -0.15633308345656335, -0.15564630866652324, -0.15353626425437816, -0.1547844818297688, 
       -0.15538986484225104, -0.15167052494689795, -0.14868695509097463, -0.14861514040232315, 
       -0.15188442283612708, -0.15251549676939935, -0.153599294588957, -0.15507196712521656, 
       -0.15592721175194144, -0.1586298157054136][:16]
F1 = [0.6959071067570164, 0.7307927546459656, 0.7410932220234546, 0.7533179132935328, 
      0.7619985710883543, 0.7645716278311145, 0.7670189931472592, 0.7661391240338609, 
      0.7661391240338609, 0.7715647347693169, 0.7724421209858103, 0.7692101643752349, 
      0.7692101643752349, 0.7692101643752349, 0.7659901074535221, 0.7659901074535221, 
      0.7659901074535221, 0.7659901074535221, 0.7659901074535221, 0.7692101643752349, 
      0.7692101643752349, 0.7692101643752349][:16]
    
df_flrain = pd.DataFrame({
    "Complain": np.array(AQs),
    "F1": np.array(F1),
    "K": list(range(0, K + step_size, step_size))[:16],
    "Method": np.repeat("LC_Rain", len(AQs)),
})
alt.Chart(pd.concat([df_flrain])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_flrain.shape[0], grid=False)),
    alt.Y("Complain:Q", scale=alt.Scale(domain=[min(AQs),max(AQs)])),
    color = "Method"
)

In [ ]:
alt.Chart(pd.concat([df_flrain])).mark_line().encode(
    alt.X('K:Q', axis=alt.Axis(tickCount=df_flrain.shape[0], grid=False)),
    alt.Y("F1:Q", scale=alt.Scale(domain=[0.65, 0.85])),
    color = "Method"
)

In [ ]:
from mlsql.lc_protocol import report, predict, hessian
predict(manager_a, manager_b, proc.x_a_test, proc.x_b_test)

In [ ]:
np.where(manager_a.delta.numpy() == 0)

In [ ]:
np.where(manager_b.delta.numpy() == 0)

In [ ]:
np.where(manager.delta.numpy() == 0)

In [ ]:
from models.linear_comb import LinearComb
from mlsql.utils.utils import pack
len(pack(manager_a.variables))

In [ ]:
manager0_b.egrads().shape[1]